<a href="https://colab.research.google.com/github/baileysmoko/Fabric/blob/main/Top_100_coins.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install yfinance requests

import requests
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from google.colab import files
import time


In [ ]:
import time
import requests
import pandas as pd
from datetime import datetime, timedelta

API_KEY = "xxx"
headers = {"X-CMC_PRO_API_KEY": API_KEY}

def safe_request(url, params):
    while True:
        r = requests.get(url, headers=headers, params=params)
        data = r.json()
        if "status" in data and data["status"]["error_code"] == 1008:
            print("⏳ Hit rate limit. Sleeping 60s...")
            time.sleep(60)
            continue
        return data

def test_single_date(date):
    url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/historical"
    params = {"date": date.strftime("%Y-%m-%d"), "limit": 10, "convert": "USD"}  # Smaller limit for testing
    data = safe_request(url, params)

    if "data" in data and data["data"]:
        return True
    return False

def find_historical_limit():
    """Binary search to find the exact historical data limit"""
    today = datetime.today()

    # Start with a range we know: 10 days works, 60 days doesn't
    min_days = 10   # We know this works
    max_days = 60   # We know this doesn't work

    print("🔍 Finding your exact historical data limit...")

    while max_days - min_days > 1:
        mid_days = (min_days + max_days) // 2
        test_date = today - timedelta(days=mid_days)

        print(f"Testing {mid_days} days ago ({test_date.strftime('%Y-%m-%d')})...")

        if test_single_date(test_date):
            print(f"✅ {mid_days} days works")
            min_days = mid_days
        else:
            print(f"❌ {mid_days} days doesn't work")
            max_days = mid_days

        time.sleep(1)  # Be nice to the API

    print(f"\n🎯 Your historical data limit is approximately {min_days} days")
    print(f"📅 You can safely collect data from {(today - timedelta(days=min_days)).strftime('%Y-%m-%d')} to today")

    return min_days

# Find your exact limit
limit_days = find_historical_limit()

# Update your original code with the correct date range
print(f"\n💡 Update your code with:")
print(f"START_DATE = END_DATE - timedelta(days={limit_days})")

🔍 Finding your exact historical data limit...
Testing 35 days ago (2025-07-31)...
❌ 35 days doesn't work
Testing 22 days ago (2025-08-13)...
✅ 22 days works
Testing 28 days ago (2025-08-07)...
✅ 28 days works
Testing 31 days ago (2025-08-04)...
✅ 31 days works
Testing 33 days ago (2025-08-02)...
❌ 33 days doesn't work
Testing 32 days ago (2025-08-03)...
❌ 32 days doesn't work

🎯 Your historical data limit is approximately 31 days
📅 You can safely collect data from 2025-08-04 to today

💡 Update your code with:
START_DATE = END_DATE - timedelta(days=31)


In [ ]:
import time
import requests
import pandas as pd
from datetime import datetime, timedelta, timezone

API_KEY = "xxx"
headers = {"X-CMC_PRO_API_KEY": API_KEY}

# --- Safe request to handle rate limits ---
def safe_request(url, params):
    while True:
        r = requests.get(url, headers=headers, params=params)
        data = r.json()
        # Error code 1008 = rate limit
        if "status" in data and data["status"]["error_code"] == 1008:
            print("⏳ Hit rate limit. Sleeping 60s...")
            time.sleep(60)
            continue
        return data

# --- Collect top 100 coins for each day in date range ---
def fetch_top_100_last_n_days(n_days=31):
    all_coins = set()
    today = datetime.now(timezone.utc)
    start_date = today - timedelta(days=n_days)

    for i in range(n_days):
        date = start_date + timedelta(days=i)
        params = {
            "date": date.strftime("%Y-%m-%d"),
            "limit": 100,
            "convert": "USD"
        }
        url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/historical"
        data = safe_request(url, params)

        if "data" in data and isinstance(data["data"], list):
            for coin in data["data"]:
                all_coins.add((coin["name"], coin["symbol"]))

        print(f"✅ Processed {date.strftime('%Y-%m-%d')}")
        time.sleep(1)  # Be kind to API

    return all_coins

# --- Fetch and save ---
top_coins = fetch_top_100_last_n_days(n_days=31)
df = pd.DataFrame(list(top_coins), columns=["Name", "Symbol"])
df.sort_values("Name", inplace=True)
df.reset_index(drop=True, inplace=True)
df.to_csv("top_100_coins_last_31_days.csv", index=False)

print(f"\n🎉 Saved {len(df)} unique coins to CSV!")
df.head()


✅ Processed 2025-08-04
✅ Processed 2025-08-05
✅ Processed 2025-08-06
✅ Processed 2025-08-07
✅ Processed 2025-08-08
✅ Processed 2025-08-09
✅ Processed 2025-08-10
✅ Processed 2025-08-11
✅ Processed 2025-08-12
✅ Processed 2025-08-13
✅ Processed 2025-08-14
✅ Processed 2025-08-15
✅ Processed 2025-08-16
✅ Processed 2025-08-17
✅ Processed 2025-08-18
✅ Processed 2025-08-19
✅ Processed 2025-08-20
✅ Processed 2025-08-21
✅ Processed 2025-08-22
✅ Processed 2025-08-23
✅ Processed 2025-08-24
✅ Processed 2025-08-25
✅ Processed 2025-08-26
✅ Processed 2025-08-27
✅ Processed 2025-08-28
✅ Processed 2025-08-29
✅ Processed 2025-08-30
✅ Processed 2025-08-31
✅ Processed 2025-09-01
✅ Processed 2025-09-02
✅ Processed 2025-09-03

🎉 Saved 109 unique coins to CSV!


,Name,Symbol
0,AB,AB
1,Aave,AAVE
2,Aerodrome Finance,AERO
3,Algorand,ALGO
4,Aptos,APT
